In [2]:
 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn import tree
import graphviz 
import os
from sklearn import preprocessing 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import warnings




In [3]:
# Import Indicators Data
df = pd.read_csv("Regime-Classification-inputs copy.csv")
df.index = pd.to_datetime(df.Date) 
del df['Date']


In [4]:
# Only use dataset before 2011-05-01

df = df[df.index <= '2011-05-01']
df

,INDPRO,PCE,PMI,T10YFF,Composite Leading Indicator (CLI),New Privately-Owned Housing Units Started: Total Units,Index of Consumer Sentiment,Chicago Fed National Activity Index,Regime
Date,,,,,,,,,
1978-01-01,47.9744,1329.5,59.8,1.22,103.3894,1718,83.7,-1.21,Slowdown
1978-02-01,48.2322,1355.1,57.4,1.23,103.2654,1738,84.3,0.99,Slowdown
1978-03-01,49.1693,1377.5,55.9,1.23,103.1583,2032,78.8,1.27,Slowdown
1978-04-01,50.1546,1396.4,55.0,1.29,103.0879,2197,81.6,1.72,Slowdown
1978-05-01,50.3971,1412.0,57.7,0.98,103.0860,2075,82.9,0.35,Slowdown
...,...,...,...,...,...,...,...,...,...
2011-01-01,93.4768,10435.5,57.5,3.22,100.8496,630,74.2,0.04,Expansion
2011-02-01,93.0617,10470.1,59.0,3.42,100.8136,517,77.5,-0.39,Expansion
2011-03-01,94.0244,10550.5,59.3,3.28,100.7231,600,67.5,0.39,Expansion


In [5]:
# Change Non-Stationary to Stationary
cols = ['INDPRO', 
        'PCE',
        'New Privately-Owned Housing Units Started: Total Units' ]

non_stat = df[cols]
stat = non_stat.diff()[1:]
df = df.copy()[1:]
df[cols] = stat
df

,INDPRO,PCE,PMI,T10YFF,Composite Leading Indicator (CLI),New Privately-Owned Housing Units Started: Total Units,Index of Consumer Sentiment,Chicago Fed National Activity Index,Regime
Date,,,,,,,,,
1978-02-01,0.2578,25.6,57.4,1.23,103.2654,20.0,84.3,0.99,Slowdown
1978-03-01,0.9371,22.4,55.9,1.23,103.1583,294.0,78.8,1.27,Slowdown
1978-04-01,0.9853,18.9,55.0,1.29,103.0879,165.0,81.6,1.72,Slowdown
1978-05-01,0.2425,15.6,57.7,0.98,103.0860,-122.0,82.9,0.35,Slowdown
1978-06-01,0.3034,13.8,60.2,0.84,103.1671,-5.0,80.0,0.77,Slowdown
...,...,...,...,...,...,...,...,...,...
2011-01-01,-0.1724,43.4,57.5,3.22,100.8496,91.0,74.2,0.04,Expansion
2011-02-01,-0.4151,34.6,59.0,3.42,100.8136,-113.0,77.5,-0.39,Expansion
2011-03-01,0.9627,80.4,59.3,3.28,100.7231,83.0,67.5,0.39,Expansion


In [6]:
df = df.shift(-1)
df.drop(df.tail(1).index,inplace=True)
df

,INDPRO,PCE,PMI,T10YFF,Composite Leading Indicator (CLI),New Privately-Owned Housing Units Started: Total Units,Index of Consumer Sentiment,Chicago Fed National Activity Index,Regime
Date,,,,,,,,,
1978-02-01,0.9371,22.4,55.9,1.23,103.1583,294.0,78.8,1.27,Slowdown
1978-03-01,0.9853,18.9,55.0,1.29,103.0879,165.0,81.6,1.72,Slowdown
1978-04-01,0.2425,15.6,57.7,0.98,103.0860,-122.0,82.9,0.35,Slowdown
1978-05-01,0.3034,13.8,60.2,0.84,103.1671,-5.0,80.0,0.77,Slowdown
1978-06-01,-0.0493,1.0,60.5,0.84,103.2544,22.0,82.4,-0.10,Slowdown
...,...,...,...,...,...,...,...,...,...
2010-12-01,-0.1724,43.4,57.5,3.22,100.8496,91.0,74.2,0.04,Expansion
2011-01-01,-0.4151,34.6,59.0,3.42,100.8136,-113.0,77.5,-0.39,Expansion
2011-02-01,0.9627,80.4,59.3,3.28,100.7231,83.0,67.5,0.39,Expansion


In [7]:
# Convert category variables to dummy variables

df['Regime'] = df['Regime'].replace({'Slowdown':1,'Contraction':2, 'Expansion':3, 'Recovery':4})
df

,INDPRO,PCE,PMI,T10YFF,Composite Leading Indicator (CLI),New Privately-Owned Housing Units Started: Total Units,Index of Consumer Sentiment,Chicago Fed National Activity Index,Regime
Date,,,,,,,,,
1978-02-01,0.9371,22.4,55.9,1.23,103.1583,294.0,78.8,1.27,1
1978-03-01,0.9853,18.9,55.0,1.29,103.0879,165.0,81.6,1.72,1
1978-04-01,0.2425,15.6,57.7,0.98,103.0860,-122.0,82.9,0.35,1
1978-05-01,0.3034,13.8,60.2,0.84,103.1671,-5.0,80.0,0.77,1
1978-06-01,-0.0493,1.0,60.5,0.84,103.2544,22.0,82.4,-0.10,1
...,...,...,...,...,...,...,...,...,...
2010-12-01,-0.1724,43.4,57.5,3.22,100.8496,91.0,74.2,0.04,3
2011-01-01,-0.4151,34.6,59.0,3.42,100.8136,-113.0,77.5,-0.39,3
2011-02-01,0.9627,80.4,59.3,3.28,100.7231,83.0,67.5,0.39,3


In [8]:
# Seperate data with features and labels
X = df[df.columns[0:8]] #features
y = df['Regime']  #labels
cols = list(X.columns)

# Split the dataframe into random train and test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=47) #split by half


In [19]:
models = {
    'GaussianNB': GaussianNB(),
    'BernoulliNB': BernoulliNB(),
    'RandomForestClassifier': RandomForestClassifier(),
    'SupportVectorMachine': SVC(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'Stochastic Gradient Descent':  SGDClassifier(max_iter=5000, random_state=0),
    'Neural Nets': MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5000, 10), random_state=1),
    'XGBClassifier': XGBClassifier()
}

modelNames = ["GaussianNB","MultinomialNB",'BernoulliNB','LogisticRegression','RandomForestClassifier','SupportVectorMachine',
             'DecisionTreeClassifier', 'KNeighborsClassifier','GradientBoostingClassifier',
             'Stochastic Gradient Descent', 'Neural Nets', 'XGBClassifier']

trainScores = []
testScores = []

for m in models:
  model = models[m]
  model.fit(X_train, y_train)
    
  print(f'{m}') 
  train_score = model.score(X_train, y_train)
  print(f'Train score of trained model: {train_score*100}')
  trainScores.append(train_score*100)

  test_score = model.score(X_test, y_test)
  print(f'Test score of trained model: {test_score*100}')
  testScores.append(test_score*100)
  print(" ")
    
  y_predictions = model.predict(X_test)
  conf_matrix = confusion_matrix(y_predictions, y_test)




  print(f'Confussion Matrix: \n{conf_matrix}\n')

  predictions = model.predict(X_test)
  cm = confusion_matrix(predictions, y_test)

  tn = conf_matrix[0,0]
  fp = conf_matrix[0,1]
  tp = conf_matrix[1,1]
  fn = conf_matrix[1,0]
  accuracy  = (tp + tn) / (tp + fp + tn + fn)
  precision = tp / (tp + fp)
  recall    = tp / (tp + fn)
  f1score  = 2 * precision * recall / (precision + recall)
  specificity = tn / (tn + fp)
  print(f'Accuracy : {accuracy}')
  print(f'Precision: {precision}')
  print(f'Recall   : {recall}')
  print(f'F1 score : {f1score}')
  print(f'Specificity : {specificity}')
  print("") 
  print(f'Classification Report: \n{classification_report(predictions, y_test)}\n')
  print("")
    
    
  for m in range (1):
    current = modelNames[m]
    modelNames.remove(modelNames[m])

    preds = model.predict(X_test)
    confusion_matr = confusion_matrix(y_test, preds) #normalize = 'true'
    print("############################################################################")
    print("")
    print("")
    print("")


GaussianNB
Train score of trained model: 60.62499999999999
Test score of trained model: 53.75
 
Confussion Matrix: 
[[15  0  9  0]
 [ 1 12  2  1]
 [18  0 13  1]
 [ 0  2  3  3]]

Accuracy : 0.9642857142857143
Precision: 1.0
Recall   : 0.9230769230769231
F1 score : 0.9600000000000001
Specificity : 1.0

Classification Report: 
              precision    recall  f1-score   support

           1       0.44      0.62      0.52        24
           2       0.86      0.75      0.80        16
           3       0.48      0.41      0.44        32
           4       0.60      0.38      0.46         8

    accuracy                           0.54        80
   macro avg       0.59      0.54      0.55        80
weighted avg       0.56      0.54      0.54        80



############################################################################



BernoulliNB
Train score of trained model: 52.5
Test score of trained model: 38.75
 
Confussion Matrix: 
[[ 4  1  7  2]
 [10 13  6  2]
 [20  0 14  1]
 [ 0  0 

D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


RandomForestClassifier
Train score of trained model: 100.0
Test score of trained model: 62.5
 
Confussion Matrix: 
[[19  0  8  2]
 [ 1 14  3  1]
 [14  0 16  1]
 [ 0  0  0  1]]

Accuracy : 0.9705882352941176
Precision: 1.0
Recall   : 0.9333333333333333
F1 score : 0.9655172413793104
Specificity : 1.0

Classification Report: 
              precision    recall  f1-score   support

           1       0.56      0.66      0.60        29
           2       1.00      0.74      0.85        19
           3       0.59      0.52      0.55        31
           4       0.20      1.00      0.33         1

    accuracy                           0.62        80
   macro avg       0.59      0.73      0.58        80
weighted avg       0.67      0.62      0.64        80



############################################################################



SupportVectorMachine
Train score of trained model: 45.9375
Test score of trained model: 40.0
 
Confussion Matrix: 
[[ 3  0  0  1]
 [ 0  3  1  0]
 [31 11 26  4

D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Test score of trained model: 43.75
 
Confussion Matrix: 
[[17  4 13  3]
 [ 1  7  3  0]
 [16  2 11  2]
 [ 0  1  0  0]]

Accuracy : 0.8275862068965517
Precision: 0.6363636363636364
Recall   : 0.875
F1 score : 0.7368421052631579
Specificity : 0.8095238095238095

Classification Report: 
              precision    recall  f1-score   support

           1       0.50      0.46      0.48        37
           2       0.50      0.64      0.56        11
           3       0.41      0.35      0.38        31
           4       0.00      0.00      0.00         1

    accuracy                           0.44        80
   macro avg       0.35      0.36      0.35        80
weighted avg       0.46      0.44      0.45        80



############################################################################



GradientBoostingClassifier
Train score of trained model: 100.0
Test score of trained model: 65.0
 
Confussion Matrix: 
[[20  0 10  1]
 [ 1 14  1  1]
 [13  0 16  1]
 [ 0  0  0  2]]

Accuracy : 0.9714

D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
D:\Anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your la

Neural Nets
Train score of trained model: 73.75
Test score of trained model: 46.25
 
Confussion Matrix: 
[[16  1 14  3]
 [ 0  9  0  0]
 [17  1 12  2]
 [ 1  3  1  0]]

Accuracy : 0.9615384615384616
Precision: 0.9
Recall   : 1.0
F1 score : 0.9473684210526316
Specificity : 0.9411764705882353

Classification Report: 
              precision    recall  f1-score   support

           1       0.47      0.47      0.47        34
           2       0.64      1.00      0.78         9
           3       0.44      0.38      0.41        32
           4       0.00      0.00      0.00         5

    accuracy                           0.46        80
   macro avg       0.39      0.46      0.41        80
weighted avg       0.45      0.46      0.45        80



############################################################################



[22:56:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with

In [10]:
import xgboost as xgb

# parameter grid for our search


# initiating our gradient boosted tree

xgb = XGBClassifier(learning_rate=0.1,# Fix learning rate
                    n_estimators=1000, # Fix number of estimators
                    max_depth=4,
                    min_child_weight=4,
                    gamma=0,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    objective='multi:softmax',
                    nthread=4,
                    num_class=9,
                    reg_alpha = 0,
                    seed=47)

xgb.fit(X_train, y_train)                       

# fitting the model
best_xgb = xgb.fit(X_train, y_train)


D:\Anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:50:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:50:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [11]:

# parameter grid for our search
parameters = {
    'n_estimators': [75, 100, 125, ],
    'max_depth': [10, 50, 100, None],
    'max_features': [0.5, 1],
    'min_samples_split': [2, 3, 10],
    'min_samples_leaf': [2, 3]
}

rfc = RandomForestClassifier(criterion='gini',
                             bootstrap=False,
                             random_state=47)
classifier = GridSearchCV(estimator=rfc,
                          param_grid=parameters,
                          cv=5)

# fitting the model
best_rfc = classifier.fit(X_train, y_train)



In [12]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

# parameter grid for our search
parameters = {
    'C': [0.1, 1, 10],
    'degree': [2, 3, 4], # only applies to kernel == 'poly'
    'kernel': ('linear', 'rbf', 'poly')
}

# initiating our support vector machine
svm = svm.SVC(gamma='scale',
              probability=True,
              random_state=47)
classifier = GridSearchCV(estimator=svm,
                          param_grid=parameters,
                          cv=5)

# fitting the model
best_svm = classifier.fit(X_train, y_train)



In [13]:
#findings of each classifier passed into Voting Classifier 
#predicts the output class based on the highest majority of voting

from sklearn.ensemble import VotingClassifier

# configuring 
vc = VotingClassifier(
    
    # all classifiers we just gridsearched
    estimators=[
        ('svm', best_svm),
        ('xgb', best_xgb),   
        ('rfc', best_rfc)
    ], 
    
    # soft = probabilities are taken into account
    voting='soft',
    
    # you can add weights, as well. E.g.
    weights=[2,3,2] 
    
)

# fitting
vc = vc.fit(X_train, y_train)

import sklearn.metrics as metrics


# accuracy on test set
y_pred_vc = vc.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred_vc)

# how often is the classifier correct?
print(f'Accuracy on the test set: {accuracy*100:,.2f}%')

D:\Anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:42:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy on the test set: 62.50%


In [21]:
from sklearn.metrics import classification_report
print(f'Classification Report: \n{classification_report(y_pred_vc, y_test)}\n')

Classification Report: 
              precision    recall  f1-score   support

           1       0.53      0.57      0.55        30
           2       1.00      0.73      0.85        15
           3       0.62      0.56      0.59        32
           4       0.38      1.00      0.55         3

    accuracy                           0.61        80
   macro avg       0.63      0.72      0.63        80
weighted avg       0.65      0.61      0.62        80




In [14]:
y_pred_vc

array([3, 1, 1, 3, 3, 1, 2, 3, 1, 4, 1, 2, 3, 3, 3, 1, 2, 1, 2, 3, 3, 1,
       1, 3, 3, 3, 2, 1, 1, 3, 3, 3, 1, 3, 3, 2, 1, 4, 3, 1, 1, 2, 3, 3,
       4, 2, 1, 2, 2, 1, 1, 3, 2, 1, 2, 1, 1, 3, 3, 1, 1, 3, 1, 3, 3, 3,
       3, 2, 2, 3, 3, 3, 2, 3, 3, 2, 3, 1, 1, 1], dtype=int64)

In [19]:
#findings of each classifier passed into Voting Classifier 
#predicts the output class based on the highest majority of voting

from sklearn.ensemble import VotingClassifier

# configuring 
vc = VotingClassifier(
    
    # all classifiers we just gridsearched
    estimators=[#
        ('svm', best_svm),
        ('xgb', best_xgb),   
        ('rfc', best_rfc)
    ], 
    
    # soft = probabilities are taken into account
    # hard = only the outcome counts
    voting='hard'
    
    # you can add weights, as well. E.g.
    
    
)

# fitting
vc = vc.fit(X_train, y_train)

import sklearn.metrics as metrics


# accuracy on test set
y_pred_vc = vc.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred_vc)

# how often is the classifier correct?
print(f'Accuracy on the test set: {accuracy*100:,.2f}%')

D:\Anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:44:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy on the test set: 61.25%
